In [1]:
import pandas as pd
import numpy as np
import math


In [2]:
amazon = pd.read_csv(r'D:\IMARTICUS\Python\Model Building\NLP\Data_set\AmazonFoodReviews.csv', encoding = 'utf-8')

In [3]:
amazon.shape

(568454, 10)

In [4]:
amazon.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [5]:
amazon = amazon.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time','Summary'], axis = 1)

In [6]:
amazon.head(2)

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...


In [7]:
amazon.isnull().sum()

Score    0
Text     0
dtype: int64

In [ ]:
amazon.Score.replace({2:1,3:2,4:3,5:3},inplace =True)

In [8]:
amazon.Text = amazon.Text.str.lower()

In [9]:
from nltk.corpus import stopwords



In [10]:
l1 = stopwords.words('english')

In [11]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def text_process(mess):            ### creating a function
    """                                                        ## a docstring
    1. remove the punctuation
    2. remove the stopwords
    3. return the list of clean textwords
    
    """
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = "".join(nopunc)
    
    return [ word for word in nopunc.split() if word not in l1]

In [13]:
# amazon['Text'].apply(text_process)

In [14]:
#TDM

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
amazon_count = CountVectorizer(analyzer= text_process).fit(amazon['Text'])

In [17]:
len(amazon_count.vocabulary_ )

240626

In [18]:
df_x = amazon_count.transform(amazon['Text'])

# create the tdm

In [19]:
df_x.shape

(568454, 240626)

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_x, amazon.Score, test_size = .2)

In [21]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(454763, 240626) (454763,)
(113691, 240626) (113691,)


In [22]:
# build the NB model on train

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier


nb = MultinomialNB()
dt = DecisionTreeClassifier(criterion='entropy',max_depth= 10)


In [35]:
dt.fit(x_train , y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [30]:
nb.fit(x_train, y_train)

MultinomialNB()

In [31]:
pred_nb =  nb.predict(x_test)

In [36]:
pred_dt =  dt.predict(x_test)

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
tab_nb = confusion_matrix(y_test , pred_nb)
tab_nb

array([[ 6593,   346,   559,   593,  2403],
       [ 1366,   945,   770,   890,  2067],
       [  987,   149,  2077,  2056,  3224],
       [  635,    60,   488,  5932,  8817],
       [ 1462,   167,   506,  4532, 66067]], dtype=int64)

In [39]:
tab_dt = confusion_matrix(y_test , pred_dt)
tab_dt

array([[ 1336,     6,    22,   150,  8980],
       [  383,   115,    43,   229,  5268],
       [  261,     5,   234,   501,  7492],
       [  141,    14,    60,  1542, 14175],
       [  535,    24,    65,   937, 71173]], dtype=int64)

In [40]:
print(tab_nb.diagonal().sum()*100 / tab_nb.sum(), ' ', tab_dt.diagonal().sum()*100 / tab_dt.sum())

71.785805384771   65.44053619019975


In [ ]:
import timeit

start = timeit.default_timer()  # time start

amazon_count = CountVectorizer(analyzer= text_process).fit(amazon['Text'])

stop = timeit.default_timer()  # timer stop

execution_time = stop - start

print('program Executed in ', execution_time )

In [ ]:
# do analytics